In [1]:
!pip install -q flwr[simulation] torch torchvision matplotlib

In [2]:
from collections import OrderedDict
from typing import List, Tuple

import flwr as fl
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}")

Training on cpu using PyTorch 1.13.1+cu117 and Flower 1.2.0


## Data Preparation


In [3]:
# parameters
time_step = 48
BATCH_SIZE = 32

In [4]:
data = []
with open ('House_30.txt', 'r') as reader:
  for line in reader:
    stripped_line = line.strip().split()
    data.append(stripped_line)

tem = [x[0] for x in data]
houses = list(set(tem))

date = []
consumption = []
for i in houses:
  date.append([float(x[1]) for x in data if x[0]==i])
  consumption.append([float(x[2]) for x in data if x[0]==i])    

In [5]:
def create_label(data, time_step):
  x_nest, y_nest = [], []
  for j in range(len(data)):
    x_data, y_data = [], []
    for i in range(len(data[j]) - time_step):
      x = data[j][i: (i + time_step)]
      x_data.append(x)
      y = [data[j][i + time_step]]
      y_data.append(y)

    #x_data = np.array(x_data)[:, :, np.newaxis]
    x_data = np.array(x_data)[:, :]
    #x_data = np.array(x_data)[:, np.newaxis, :]
    x_nest.append(x_data)
    y_nest.append(y_data)
  x_nest = np.array(x_nest)
  y_nest = np.array(y_nest)
  return x_nest, y_nest
# 可能要去掉x的最后一个维度 从（48，1）变为（48）

In [6]:
input, labels = create_label(consumption, time_step)


In [7]:
input = np.float32(input)
labels = np.float32(labels)


### Create Dataset

In [8]:
# 定义GetLoader类，继承Dataset方法，并重写__getitem__()和__len__()方法
class GetLoader(torch.utils.data.Dataset):
	# 初始化函数，得到数据
    def __init__(self, data_root, data_label):
        self.data = data_root
        self.label = data_label
    # index是根据batchsize划分数据后得到的索引，最后将data和对应的labels进行一起返回
    def __getitem__(self, index):
        data = self.data[index]
        labels = self.label[index]
        return data, labels
    # 该函数返回数据大小长度，目的是DataLoader方便划分，如果不知道大小，DataLoader会一脸懵逼
    def __len__(self):
        return len(self.data)


In [9]:
length = len(input[0])
val = int(0.7*length)
test = int(0.9*length)
trainloaders = []
valloaders = []
testloaders = []

def load_datasets(input, labels):

  Xtrain_raw = [x[0: val] for x in input]
  Xval_raw = [x[val: test] for x in input]
  Xtest_raw = [x[test: ] for x in input]

  Ytrain_raw = [x[0: val] for x in labels]
  Yval_raw = [x[val: test] for x in labels]
  Ytest_raw = [x[test: ] for x in labels]

  for i in range(30):
    ds_train = GetLoader(Xtrain_raw[i], Ytrain_raw[i])
    trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True, drop_last=True))
    ds_val = GetLoader(Xval_raw[i], Yval_raw[i])
    valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE, drop_last=True))
    ds_test= GetLoader(Xtest_raw[i], Ytest_raw[i])
    testloaders.append(DataLoader(ds_test, batch_size=BATCH_SIZE, drop_last=True))

  return trainloaders, valloaders, testloaders




In [10]:
trainloaders, valloaders, testloaders = load_datasets(input, labels)

## Create Model

In [11]:
import torch.nn as nn


In [12]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        act = nn.Sigmoid
        self.body = nn.Sequential(
            nn.Linear(48, 48),
            act(),
            nn.Linear(48, 48),
            act(),
            nn.Linear(48, 1)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.body(x)
        return out

criterion = nn.MSELoss() 
net = DNN().to(DEVICE)


In [13]:
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for x, y in trainloader:
            
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()

            #x = x.unsqueeze(0)
            outputs = net(x)
            loss = criterion(net(x), y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss
           
        epoch_loss /= len(trainloader.dataset)
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}") 


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = nn.MSELoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for x, y in testloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = net(x)
            loss += criterion(outputs, y).item()

    loss /= len(testloader.dataset)
   
    return loss 

### central test

In [14]:
x, y = next(iter(trainloaders[0]))

In [15]:
net = DNN().to(DEVICE)
client_n = 30
MSE_cen = [0 for i in range(client_n)]
epoch = 300

for i in range(client_n):
  trainloader = trainloaders[i]
  testloader = testloaders[i]
  train(net, trainloader, epoch)
  loss = test(net, testloader)
  print(f"client {i}: centralized model loss {loss}")
  MSE_cen[i] = loss


client 0: centralized model loss 0.001308749587187323
client 1: centralized model loss 0.0013043681227433788
client 2: centralized model loss 0.0008208717107468189
client 3: centralized model loss 0.00046298728983434396
client 4: centralized model loss 0.000538724139325518
client 5: centralized model loss 0.000885935108223785
client 6: centralized model loss 0.0010877334485751666
client 7: centralized model loss 0.0016185651102460154


KeyboardInterrupt: 

In [ ]:
trainloader = trainloaders[0]
valloader = valloaders[0]
testloader = testloaders[0]


net = DNN().to(DEVICE)

for epoch in range(100):
    train(net, trainloader, 300)
    loss = test(net, valloader)
    print(f"Epoch {epoch+1}: validation loss {loss}")

loss = test(net, testloader)
print(f"Final test set performance:\n\tloss {loss}")



/usr/local/lib/python3.8/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1: validation loss 0.004740860300121185
Epoch 2: validation loss 0.004740860321881058
Epoch 3: validation loss 0.0047408603291343495
Epoch 4: validation loss 0.004740860362499488
Epoch 5: validation loss 0.004740860347992906
Final test set performance:
	loss 0.003996639648841178
